In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


In [2]:
train=pd.read_csv('train_128.csv',index_col=0)

In [3]:
from collections import Counter
# 查看所生成的样本类别分布
x,y = train.iloc[:,0:train.shape[1]-2],train.iloc[:,train.shape[1]-1]
print(Counter(y))

Counter({0: 4524, 1: 774})


In [4]:
x=train.iloc[:,:-1]
x=np.array(x)
y=train.iloc[:,-1]
y=np.array(y)

In [5]:
# 使用imlbearn库中上采样方法中的SMOTE接口
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
# 定义SMOTE模型，random_state相当于随机数种子的作用
smo = SMOTE(random_state=42)
X,Y = smo.fit_sample(x, y)
print(Counter(Y))
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=0)
print(len(x_train))
print(len(x_test))

Counter({0: 4524, 1: 4524})
6333
2715


In [6]:
# 定义ks评分
def KS(Y,Y_hat):
    data = {"Y":Y,"Y_hat":Y_hat}
    df = pd.DataFrame(data)
    bins = np.array([-0.1,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
    category = pd.cut(df["Y_hat"],bins=bins)
    category = category.sort_values()
    #max_index = len(np.unique(df["Y_hat"]))
    Y = df.ix[category.index,:]['Y']
    Y_hat = df.ix[category.index,:]['Y_hat']
    df2 = pd.concat([Y,Y_hat],axis=1)
    df3 = pd.pivot_table(df2,values = ['Y_hat'],index ='Y_hat',columns='Y',aggfunc=len,fill_value=0)
    df4 = np.cumsum(df3)
    df5 = df4/df4.iloc[:,1].max()
    ks = max(abs(df5.iloc[:,0] - df5.iloc[:,1]))
    return ks/len(bins)

In [7]:
from xgboost import XGBClassifier
from xgboost import plot_importance
model = XGBClassifier(learning_rate=0.1,
                      n_estimators=470,           # 树的个数-10棵树建立xgboost
                      max_depth=12,               # 树的深度
                      min_child_weight = 1,      # 叶子节点最小权重
                      gamma=0.3,                  # 惩罚项中叶子结点个数前的参数
                      subsample=1,               # 所有样本建立决策树
                      colsample_btree=1,         # 所有特征建立决策树
                      random_state=27,           # 随机数
                      slient = 0
                      )
model.fit(x_train,y_train)
y_pred=model.predict_proba(x_test)
Y_hat=np.array(y_pred)[:,1]
KS(y_test,Y_hat)

d:\python3.6\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
d:\python3.6\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


0.07684062437194346

In [27]:
test=pd.read_csv('test/test_128.csv',index_col=0)
test.head()

,sex_0,earning_avg,cur_bill_bal_max,last_bill_amount_max,borrow_cash_max,last_repay_amount_max,cur_bill_minrepay_max,last_repay_amount_median,last_repay_amount_mean,credit_amount_max,...,adjust_amount_sum,last_repay_amount_var - last_bill_amount_var,brows_num before lending_9,repay_status_var,repay_status_mean,avail_bal_median,avail_bal_min - borrow_cash_min,education_1,avail_bal_std,adjust_amount_mean
0,0,5.196190,3.022935e-01,4.035668e-01,5.520894e-01,3.844486e-01,2.227774e-01,-1.775067e-01,-4.062762e-01,3.470865e-01,...,-1.742170e-02,2.531970e-01,-0.115787,-2.816525e-02,-2.343536e-02,-1.263734e-01,5.987055e-01,0,2.370526,-2.441470e-02
1,0,0.817572,6.419666e-02,4.497680e-01,6.469952e-01,4.442306e-01,1.524739e-01,6.557869e-01,7.601876e-01,3.883012e-01,...,-1.742170e-02,-3.148047e-01,-0.115787,-2.816525e-02,-2.343536e-02,-1.263734e-01,5.987055e-01,0,-0.319200,-2.441470e-02
2,0,0.072266,-1.636578e-01,-2.466690e+00,-1.890427e+00,-2.613476e+00,-5.122171e-01,-1.684127e+00,-1.976238e+00,1.853878e-01,...,-1.742170e-02,3.225024e-01,-0.115787,-2.816525e-02,-2.343536e-02,-1.263734e-01,5.987055e-01,0,-0.319200,-2.441470e-02
3,0,0.072266,-5.176291e-01,2.597205e-01,6.828189e-01,2.959261e-01,-7.061846e-02,6.610157e-01,1.036403e+00,4.414360e-01,...,-1.742170e-02,-3.209859e+00,-0.115787,-2.816525e-02,-2.343536e-02,-1.263734e-01,-1.933778e+00,0,-0.319200,-2.441470e-02
4,0,-0.312316,-1.769490e-14,2.017567e-15,-2.964101e-14,6.387110e-15,2.855472e-14,1.179267e-14,-1.015493e-14,-6.168145e-14,...,1.299292e-17,-8.651885e-16,-0.115787,-5.052635e-18,-8.097164e-18,-7.499588e-17,6.644972e-15,0,0.000000,3.814837e-18


In [28]:
test.shape

(13899, 128)

In [30]:
test=np.array(test)
model.fit(x_train,y_train)
y_pred=model.predict_proba(test)

In [31]:
y_pred.shape

(13899, 2)

In [32]:
label=pd.DataFrame(y_pred,columns=['probability0','probability'])

In [33]:
label=label.drop(['probability0'],axis=1)

In [34]:
label.head()

,probability
0,0.578330
1,0.036506
2,0.748826
3,0.053070
4,0.591049


In [35]:
label.to_csv('test/tijao.csv')

In [ ]:


KS_score = make_scorer(KS, greater_is_better=True, needs_proba = True)

scoring = { 'KS': KS_score}

gsearch = GridSearchCV(estimator = bestclf, param_grid = param_test,scoring=scoring, n_jobs=1,cv=ps, refit ='AR')


In [9]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
#网格搜索进行参数调优
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},
                    {'kernel': ['poly'],'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]}]

In [27]:
from sklearn.svm import SVC
#调用SVC()
clf = GridSearchCV(SVC(),tuned_parameters, n_jobs=1,cv=5, refit ='KS')
#fit()训练
clf.fit(x_train,y_train)
"""y_pred=model.predict_proba(x_test)
Y_hat=np.array(y_pred)[:,1]
KS(y_test,Y_hat)"""
#print(clf.grid_scores_, clf.best_params_,clf.best_score_)
best_index_ks = np.nonzero(Grisearch.cv_results_['rank_test_KS'] == 1)[0][0]
best_score_ks = Gridsearch.cv_results_['mean_test_KS'][best_index_ks]

"#print(clf.grid_scores_, clf.best_params_,clf.best_score_)\nbest_index_ks = np.nonzero(Grisearch.cv_results_['rank_test_KS'] == 1)[0][0]\nbest_score_ks = Gridsearch.cv_results_['mean_test_KS'][best_index_ks]"